In [1]:
!pip3 install -Uqq conllu tabulate
# data loading
import conllu
import json

def dict_prettify(token):
    return json.dumps(token, indent=2, ensure_ascii=False)

def corpus_load(path):
    with open(path, 'r') as file:
        return conllu.parse(file.read())
    return None

corpus_train = corpus_load('train.conllu')
corpus_dev = corpus_load('dev.conllu')
print(f"SENTENCE.serialize()=\n{corpus_train[10].serialize()}")
print(f"SENTENCE[1]=\n{dict_prettify(corpus_train[10][1])}")

SENTENCE.serialize()=
# sent_id = 20000715_AFP_ARB.0075:5
# text = وتعذر لمتحدثة باسم وزارة الدفاع الالمانية ان تؤكد اليوم السبت هذه المعلومات .
# text_bw = wtE*r lmtHdvp bAsm wzArp AldfAE AlAlmAnyp An t&kd Alywm Alsbt h*h AlmElwmAt .
1-2	وتعذر	_	_	_	_	_	_	_	_
1	و	w	CCONJ	CONJ	_	2	cc	_	bw=wa
2	تعذر	taEa*~ar_1	VERB	PV+PVSUFF_SUBJ:3MS	Aspect=Perf|Gender=Masc|Mood=Ind|Number=Sing|Person=3|Voice=Act	0	root	_	bw=taEa*~ara
3-4	لمتحدثة	_	_	_	_	_	_	_	_
3	ل	l	ADP	PREP	AdpType=Prep	4	case	_	bw=li
4	متحدثة	mutaHad~iv_1	NOUN	NOUN+NSUFF_FEM_SG+CASE_INDEF_GEN	Case=Gen|Definite=Ind|Gender=Fem|Number=Sing	2	obj	_	bw=mutaHad~ivapK
5-6	باسم	_	_	_	_	_	_	_	_
5	ب	b	ADP	PREP	AdpType=Prep	6	case	_	bw=bi
6	اسم	{isom_1	NOUN	NOUN+CASE_DEF_GEN	Case=Gen|Definite=Com|Gender=Masc|Number=Sing	4	obj	_	bw={isomi
7	وزارة	wizArap_1	NOUN	NOUN+NSUFF_FEM_SG+CASE_DEF_GEN	Case=Gen|Definite=Com|Gender=Fem|Number=Sing	6	nmod:poss	_	bw=wizArapi
8	الدفاع	difAE_1	NOUN	DET+NOUN+CASE_DEF_GEN	Case=Gen|Definite=Def|Gender=Masc|Number

In [12]:
import random

def _tree_get_subsentence(node, is_root, skip_cond, subsentence):
    if skip_cond is not None and skip_cond(node):
        return
    token = node.token
    head_dist = token['head'] - token['id'] if not is_root else 0
    deprel = token['deprel'] if not is_root else 'root'
    subsentence[token['id']] = (token['id'], token['form'], token['lemma'],
                                token['upos'], token['head'], deprel, token['xpos'], 
                                token['feats'], head_dist)
    for ch in node.children:
        _tree_get_subsentence(ch, False, skip_cond, subsentence)

def tree_get_subsentence(node, skip_cond=None):
    subsentence = {}
    _tree_get_subsentence(node, True, skip_cond, subsentence)
    subsentence = sorted([(k, v) for k, v in subsentence.items()])
    if len(subsentence) > 0 and subsentence[-1][0] - subsentence[0][0] + 1 != len(subsentence):
        return None
    for (k, v) in subsentence:
        if v[0] + v[-1] < subsentence[0][0] or v[0] + v[-1] > subsentence[-1][0]:
            print(subsentence)
            return None
    return [i2 for (i1, i2) in subsentence]

def _tree_get_subsentences_of_len(node, len_range, root_cond, skip_cond, result):
    if root_cond is None or root_cond(node):
        ss = tree_get_subsentence(node, skip_cond)
        if ss is not None and len_range[0] <= len(ss) and len(ss) < len_range[1]:
            result.append(ss)
    for ch in node.children:
        _tree_get_subsentences_of_len(ch, len_range, root_cond, skip_cond, result)

def tree_get_subsentences_of_len(node, len_range, root_cond=None, skip_cond=None):
    result = []
    _tree_get_subsentences_of_len(node, len_range, root_cond, skip_cond, result)
    return result

def corpus_get_short_sentences(corpus):
    for s in corpus:
        tree = s.to_tree()
        for ss in tree_get_subsentences_of_len(
            tree, (5, 6),
            root_cond=lambda n: n.token['upos'] == 'VERB',
            skip_cond=lambda n: n.token['upos'] == 'PUNCT' or '،' in n.token['form']):
            yield ss

random.seed(1)
for ss in corpus_get_short_sentences(random.sample(corpus_dev, 100)):
    print(' '.join([f"{v[1]}" for v in ss]))
    # print(' '.join([f"{v[1]}.{v[2]}{v[3]:+d}" for v in ss]))

تشكل خطراً على جيران ها
تتخذ ها في الساعات الحرجة
ما زالت في بدايات ها
أن نقدم ل هم الدعم
افاد مراسل وكالة فرانس برس
ينفذ ها الشباب في فلسطين
يتزعم ه اسامة بن لادن
خاض ها البلدان عام 1962
تؤدي إلى مقتل مواطنين أبرياء
لا ينتمون إلى الحزب الحاكم
اذا تمكنا من تحقيق تقدم
س يضم قبرص إلي ه
يبدأ خفض الرسوم الجمركية تدريجياً


In [13]:
train = list(corpus_get_short_sentences(corpus_train))
dev = list(corpus_get_short_sentences(corpus_dev))

In [14]:
print(len(train))
print(len(dev))

2030
260


In [15]:
def sentences_to_file(filename, sentences, sep='\t'):
    with open(filename, 'w') as f:
        f.write(f"sent_id{sep}token_id{sep}form{sep}lemma{sep}upos{sep}head_id{sep}deprel{sep}xpos{sep}feats{sep}head_dist\n")
        for i, s in enumerate(sentences):
            for w in s:
                f.write(f"{i}")
                for v in w:
                    f.write(f"{sep}{v}")
                f.write("\n")

sentences_to_file('short_dev.csv', dev)
sentences_to_file('short_train.csv', train)